# ZAF202310_Book_Recommendation_Service

## Overview

## Methodology

## Business Segments

## Data
1. Book Recommendation System - [Link](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset)

## Papers
- Collaborative Filtering Recommender Systems - [Link](https://www.researchgate.net/publication/200121027_Collaborative_Filtering_Recommender_Systems)

## 1. Load Dataset

In [1]:
# Importing Libraries
import numpy as np
import pandas as pd

In [2]:
# Reading the csv files
books_df = pd.read_csv('data/Books.csv')
ratings_df = pd.read_csv('data/Ratings.csv')
user_df = pd.read_csv('data/Users.csv')

C:\Users\shubh\AppData\Local\Temp\ipykernel_7880\2526589295.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('data/Books.csv')


## 2. Data Preprocessing

In [6]:
# Merging data into one dataframe
book_ratings_df = ratings_df.merge(books_df, on='ISBN')
user_book_ratings_df = book_ratings_df.merge(user_df, on='User-ID')

In [7]:
# First look at data
user_book_ratings_df

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"cincinnati, ohio, usa",23.0
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,"cincinnati, ohio, usa",23.0
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,"cincinnati, ohio, usa",23.0
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,"cincinnati, ohio, usa",23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1031131,276442,2862749796,7,Le Huit,Katherine Neville,2002,Le Cherche Midi,http://images.amazon.com/images/P/2862749796.0...,http://images.amazon.com/images/P/2862749796.0...,http://images.amazon.com/images/P/2862749796.0...,"genève, genève, switzerland",62.0
1031132,276618,3788097000,5,Ludwig Marum: Briefe aus dem Konzentrationslag...,Ludwig Marum,1984,C.F. MÃ¼ller,http://images.amazon.com/images/P/3788097000.0...,http://images.amazon.com/images/P/3788097000.0...,http://images.amazon.com/images/P/3788097000.0...,"stuttgart, \n/a\""., germany""",NaN
1031133,276647,0553571001,0,Christmas With Anne and Other Holiday Stories:...,L. M. Montgomery,2001,Starfire,http://images.amazon.com/images/P/0553571001.0...,http://images.amazon.com/images/P/0553571001.0...,http://images.amazon.com/images/P/0553571001.0...,"arlington heights, illinois, usa",13.0
1031134,276647,0689822294,10,Heaven (Coretta Scott King Author Award Winner),Angela Johnson,1998,Simon &amp; Schuster Children's Publishing,http://images.amazon.com/images/P/0689822294.0...,http://images.amazon.com/images/P/0689822294.0...,http://images.amazon.com/images/P/0689822294.0...,"arlington heights, illinois, usa",13.0


In [39]:
# Creating a dictionary of unique Book-Titles and creating a new column
book_dict = {}
for idx, book in enumerate(user_book_ratings_df['Book-Title'].unique()):
    book_dict[book] = idx
user_book_ratings_df['Book-ID'] = user_book_ratings_df['Book-Title'].map(book_dict)

In [40]:
# Creating a dictionary of unique User-IDs and creating a new column
user_dict = {}
for idx, user in enumerate(user_book_ratings_df['User-ID'].unique()):
    user_dict[user] = idx
user_book_ratings_df['New-User-ID'] = user_book_ratings_df['User-ID'].map(user_dict)

In [44]:
# Creating Final DF to use
final_df = user_book_ratings_df[['New-User-ID', 'Book-ID', 'Book-Rating']]

In [45]:
# Final DF look
final_df

,New-User-ID,Book-ID,Book-Rating
0,0,0,0
1,1,0,5
2,1,1,9
3,1,2,8
4,1,3,9
...,...,...,...
1031131,92102,241066,7
1031132,92103,241067,5
1031133,92104,241068,0
1031134,92104,241069,10


## 3. Data Preparation

In [46]:
# Dividing data into X and y sets
X = pd.DataFrame({'user-id': final_df['New-User-ID'],
                 'book-id': final_df['Book-ID']})
y = final_df['Book-Rating']

In [64]:
# Checking X and y shape and number of unique users and books
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
n_users = X['user-id'].nunique()
n_books = X['book-id'].nunique()
print(f"Number of unique users: {n_users}")
print(f"Number of unique books: {n_books}")

X shape: (1031136, 2)
y shape: (1031136,)
Number of unique users: 92106
Number of unique books: 241071


## 4. Model Building

In [77]:
import math
from itertools import zip_longest
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.nn import functional as F 
from torch.optim.lr_scheduler import _LRScheduler

In [54]:
# Creating a class for reviews iterator
class ReviewsIterator:
    
    def __init__(self, X, y, batch_size=32, shuffle=True):
        
        X, y = np.array(X), np.array(y)
        
        if shuffle:
            index = np.random.permutation(X.shape[0])
            X, y = X[index], y[index]
            
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_batches = int(math.ceil(X.shape[0] / self.batch_size))
        self.current_ = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        if self.current_ >= self.n_batches:
            raise StopIteration()
        k = self.current_
        self.current_ += 1
        bs = self.batch_size
        return self.X[k*bs: (k+1)*bs], self.y[k*bs: (k+1)*bs]

In [52]:
# Function to yield X and y in batches
def batches(X, y, bs=32, shuffle=True):
    for xb, yb in ReviewsIterator(X, y, bs, shuffle):
        xb = torch.LongTensor(xb)
        yb = torch.FloatTensor(yb)
        yield xb, yb.view(-1, 1)

In [55]:

for x_batch, y_batch in batches(X, y, bs=4):
    print(x_batch)
    print(y_batch)
    break

tensor([[16729,  2219],
        [  485,  9416],
        [  774,  9340],
        [17401,  9217]])
tensor([[0.],
        [7.],
        [0.],
        [6.]])


In [71]:
# Creating a class for building Embedding Net
class EmbeddingNet(nn.Module):
    
    def __init__(self, n_users, n_books, 
                n_factors=50, embedding_dropout=0.02,
                hidden=10, dropouts=0.2):
        super().__init__()
        hidden = get_list(hidden)
        dropouts = get_list(dropouts)
        n_last = hidden[-1]
        
        def gen_layers(n_in):
            nonlocal hidden, dropouts
            assert len(dropouts) <= len(hidden)
            
            for n_out, rate in zip_longest(hidden, dropouts):
                yield nn.Linear(n_in, n_out)
                yield nn.ReLU()
                if rate is not None and rate > 0.:
                    yield nn.Dropout(rate)
                n_in = n_out
                
        self.u = nn.Embedding(n_users, n_factors)
        self.b = nn.Embedding(n_books, n_factors)
        self.drop = nn.Dropout(embedding_dropout)
        self.hidden = nn.Sequential(*list(gen_layers(n_factors * 2)))
        self.fc = nn.Linear(n_last, 1)
        self._init()
        
    def forward(self, users, books, minmax=None):
        features = torch.cat([self.u(users), self.b(books)], dim=1)
        x = self.drop(features)
        x = self.hidden(x)
        out = torch.sigmoid(self.fc(x))
        if minmax is not None:
            min_rating, max_rating = minmax
            out = out*(max_rating - min_rating + 1) + min_rating - 0.5
        return out
    
    def _init(self):
        def init(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)
                
        self.u.weight.data.uniform_(-0.05, 0.05)
        self.b.weight.data.uniform_(-0.05, 0.05)
        self.hidden.apply(init)
        init(self.fc)
        
def get_list(n):
    if isinstance(n, (int, float)):
        return [n]
    elif hasattr(n, '__iter__'):
        return list(n)
    raise TypeError('layers configuraiton should be a single number or a list of numbers')

In [75]:
# Creating a class for Cyclic LR Scheduler
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [76]:
# Function for calculating cosine value
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + math.cos(math.pi*t/t_max))/2
    
    return scheduler

In [78]:
# Splitting X and y into train and validation states
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
datasets = {'train': (X_train, y_train), 'val': (X_valid, y_valid)}
dataset_sizes = {'train': len(X_train), 'val': len(X_valid)}

In [79]:
# Defining min and max ratings
minmax = final_df['Book-Rating'].min(), final_df['Book-Rating'].max()
minmax

(0, 10)

In [80]:
# Initialiing the network
model = EmbeddingNet(
    n_users=n_users, n_books=n_books, 
    n_factors=150, hidden=[500, 500, 500], 
    embedding_dropout=0.05, dropouts=[0.5, 0.5, 0.25])

In [88]:
lr = 1e-3
wd = 1e-5
bs = 32 
n_epochs = 15
patience = 10
no_improvements = 0
best_loss = np.inf
best_weights = None
history = []
lr_history = []

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net.to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
iterations_per_epoch = int(math.ceil(dataset_sizes['train'] // bs))
scheduler = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/10))

for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase in ('train', 'val'):
        training = phase == 'train'
        running_loss = 0.0
        n_batches = 0
        batch_num = 0
        for batch in batches(*datasets[phase], shuffle=training, bs=bs):
            x_batch, y_batch = [b.to(device) for b in batch]
            optimizer.zero_grad()
            # compute gradients only during 'train' phase
            with torch.set_grad_enabled(training):
                outputs = model(x_batch[:, 0], x_batch[:, 1], minmax)
                loss = criterion(outputs, y_batch)
                
                # don't update weights and rates when in 'val' phase
                if training:
                    optimizer.step()
                    scheduler.step()
                    loss.backward()
                    lr_history.extend(scheduler.get_lr())
                    
            running_loss += loss.item()
            
        epoch_loss = running_loss / dataset_sizes[phase]
        stats[phase] = epoch_loss
        
        # early stopping: save weights of the best model so far
        if phase == 'val':
            if epoch_loss < best_loss:
                print('loss improvement on epoch: %d' % (epoch + 1))
                best_loss = epoch_loss
#                 best_weights = copy.deepcopy(net.state_dict())
                torch.save(model, 'model.pt')
                no_improvements = 0
            else:
                no_improvements += 1
                
    history.append(stats)
    print('[{epoch:03d}/{total:03d}] train: {train:.4f} - val: {val:.4f}'.format(**stats))
    if no_improvements >= patience:
        print('early stopping after epoch {epoch:03d}'.format(**stats))
        break

loss improvement on epoch: 1
[001/015] train: 12.0935 - val: 12.3695



KeyboardInterrupt



In [89]:
model = torch.load('model.pt')

In [90]:
model.eval()

EmbeddingNet(
  (u): Embedding(92106, 150)
  (b): Embedding(241071, 150)
  (drop): Dropout(p=0.05, inplace=False)
  (hidden): Sequential(
    (0): Linear(in_features=300, out_features=500, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=500, out_features=500, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=500, out_features=500, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.25, inplace=False)
  )
  (fc): Linear(in_features=500, out_features=1, bias=True)
)